# 베이스라인

In [1]:
import os
import json
import glob
from tqdm import tqdm
from collections import defaultdict
import shutil # [Phase 2]를 위해 추가

print("--- [Phase 1] 데이터 통합 및 정제 시작 ---")

# 경로 설정
base_dir = r"C:\Users\daboi\Desktop\ai05-level1-project"
train_img_dir = os.path.join(base_dir, "train_images")
train_ann_dir = os.path.join(base_dir, "train_annotations")

# 모든 JSON 파일 검색
json_files = glob.glob(os.path.join(train_ann_dir, "**", "*.json"), recursive=True)
print(f"총 {len(json_files)}개의 JSON 어노테이션 파일 검색")

# 데이터 통합 (1489개 이미지)
master_data = defaultdict(lambda: {
    'image_path': '', 
    'width': 0, 
    'height': 0, 
    'annotations': []
})

class_to_id = {} # YOLO 학습용 ID (0, 1, 2...)
current_id = 0
processing_errors = 0

for json_path in tqdm(json_files, desc="통합 중 (Phase 1)"):
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        img_info = data['images'][0]
        img_filename = img_info['file_name']

        if not master_data[img_filename]['image_path']:
            image_path = os.path.join(train_img_dir, img_filename)
            master_data[img_filename]['image_path'] = image_path
            master_data[img_filename]['width'] = img_info['width']
            master_data[img_filename]['height'] = img_info['height']

        category_map = {cat['id']: cat['name'] for cat in data['categories']}

        for ann in data['annotations']:
            bbox = ann['bbox'] # [x, y, w, h]
            ann_cat_id = ann['category_id']
            
            if ann_cat_id not in category_map:
                processing_errors += 1
                continue

            class_name = category_map[ann_cat_id]

            if class_name not in class_to_id:
                class_to_id[class_name] = current_id
                current_id += 1
                
            class_id = class_to_id[class_name]

            master_data[img_filename]['annotations'].append({
                'class_id': class_id,
                'class_name': class_name,
                'bbox': bbox
            })
    except Exception:
        continue

print(f"총 {len(master_data)}개의 이미지 데이터 통합 완료 (정제 전)")
print(f"총 {len(class_to_id)}개의 고유 클래스 발견")

# 오류 데이터 제거 (17개 파일)
print("\nEDA 기반 오류 파일 17개 제거 시작")
iou_error_files = [
    "K-003351-018147-020238_0_2_0_2_90_000_200.png", 
    "K-003483-027733-030308-036637_0_2_0_2_90_000_200.png",
    "K-003351-020238-031863_0_2_0_2_70_000_200.png", 
    "K-003351-029667-031863_0_2_0_2_70_000_200.png",
    "K-003483-019861-025367-029667_0_2_0_2_90_000_200.png", 
    "K-002483-003743-012081-019552_0_2_0_2_90_000_200.png",
    "K-003483-019861-020238-031885_0_2_0_2_70_000_200.png", 
    "K-003351-003832-029667_0_2_0_2_90_000_200.png",
    "K-001900-016548-019607-033009_0_2_0_2_70_000_200.png"
]
oob_error_files = [
    "K-003351-016262-018357_0_2_0_2_75_000_200.png",
    "K-003544-004543-012247-016551_0_2_0_2_70_000_200.png"
]
nexium_suspect_images = [
    'K-001900-010224-016551-031705_0_2_0_2_70_000_200.png', 
    'K-001900-010224-016551-031705_0_2_0_2_75_000_200.png',
    'K-001900-010224-016551-031705_0_2_0_2_90_000_200.png', 
    'K-001900-010224-016551-033009_0_2_0_2_70_000_200.png',
    'K-001900-010224-016551-033009_0_2_0_2_75_000_200.png', 
    'K-001900-010224-016551-033009_0_2_0_2_90_000_200.png'
]
files_to_delete = set(iou_error_files + oob_error_files + nexium_suspect_images)

deleted_count = 0
# 원본 master_data의 복사본을 순회 (순회 중 원본 삭제를 위해)
for filename in list(master_data.keys()):
    if filename in files_to_delete:
        del master_data[filename]
        deleted_count += 1

print(f"총 {deleted_count}개의 오류 파일을 master_data에서 삭제")
print(f"최종 정제된 master_data 개수: {len(master_data)}개 (1472개여야 함)")


# 최종 산출물 저장
# 정제된 1472개 데이터
clean_master_path = os.path.join(base_dir, "train_master_annotations_clean.json")
with open(clean_master_path, "w", encoding='utf-8') as f:
    json.dump(master_data, f, ensure_ascii=False, indent=4)

# 클래스 ID 맵
class_map_path = os.path.join(base_dir, "class_to_id.json")
with open(class_map_path, "w", encoding='utf-8') as f:
    json.dump(class_to_id, f, ensure_ascii=False, indent=4)

print(f"파일 저장 완료: {clean_master_path}")
print(f"파일 저장 완료: {class_map_path}")
print("--- [Phase 1] 완료 ---")

--- [Phase 1] 데이터 통합 및 정제 시작 ---
총 4526개의 JSON 어노테이션 파일 검색


통합 중 (Phase 1): 100%|██████████| 4526/4526 [00:00<00:00, 11989.87it/s]

총 1489개의 이미지 데이터 통합 완료 (정제 전)
총 73개의 고유 클래스 발견

EDA 기반 오류 파일 17개 제거 시작
총 17개의 오류 파일을 master_data에서 삭제
최종 정제된 master_data 개수: 1472개 (1472개여야 함)
파일 저장 완료: C:\Users\daboi\Desktop\ai05-level1-project\train_master_annotations_clean.json
파일 저장 완료: C:\Users\daboi\Desktop\ai05-level1-project\class_to_id.json
--- [Phase 1] 완료 ---


In [ ]:
import os
import json
import glob
from ultralytics import YOLO

# 경로 설정
base_dir = r"C:\Users\daboi\Desktop\ai05-level1-project"
combined_dataset_dir = os.path.join(base_dir, "CombinedDataset3")
yaml_path_combined = os.path.join(combined_dataset_dir, 'data_70_15_15_split.yaml')
exp_dir = os.path.join(base_dir, "Exp")
class_to_id_path = os.path.join(base_dir, "class_to_id.json")

# 파일 존재 여부 확인
if not os.path.exists(class_to_id_path):
    print(f"오류: class_to_id.json 파일을 찾을 수 없음")
    exit()
if not os.path.exists(yaml_path_combined):
    print(f"오류: data_70_15_15_split.yaml 파일을 찾을 수 없음")
    exit()

print(f"Combined 데이터셋 경로: {combined_dataset_dir}")
print(f"data.yaml 경로: {yaml_path_combined}")
print("="*50)

# Baseline 모델 학습
print("\n[YOLOv8n] '1단계: Baseline 모델' 학습 시작")
print("목표: Valid mAP75 기준 점수 확보")
print("="*50)

model = YOLO('yolov8n.pt')

results = model.train(
    data=yaml_path_combined,
    epochs=150,
    patience=30,
    imgsz=640,
    batch=16,
    device=0,
    project=exp_dir,
    name='yolo_n_baseline_SGD_lr01', 
    exist_ok=True,
    optimizer='SGD', 
    lr0=0.01, 
    augment=True, 
    workers=0
)

# 학습 완료 후 최종 mAP75 확인
print("\n'1단계: Baseline' 학습 완료")

# ❗️ 이 섹션이 학습이 "모두 끝난 후" 자동 실행되어
# ❗️ 'best.pt' 모델의 "최종 점수"를 출력합니다.
print("\n최종 검증 수행")
best_model_path = os.path.join(exp_dir, 'yolo_n_baseline_SGD_lr01', 'weights', 'best.pt')

if os.path.exists(best_model_path):
    print(f"'{best_model_path}' 에서 best.pt 모델을 로드")
    best_model = YOLO(best_model_path)
    
    # best.pt 모델로 'val' 데이터셋에 대한 최종 평가 실행
    val_results = best_model.val(
        data=yaml_path_combined,
        split='val',
        verbose=True  # True로 설정하여 모든 평가 지표가 콘솔에 보이도록 함
    )
    
    print("\n" + "="*50)
    print("최종 검증 결과 (Best Model)")
    print(f"mAP50: {val_results.box.map50:.4f}")
    print(f"mAP75: {val_results.box.map75:.4f}")  # 팀 규칙에 필요한 최종 점수
    print(f"mAP50-95: {val_results.box.map:.4f}")
    print("="*50)
else:
    print(f"오류: 최적 모델 가중치 '{best_model_path}'를 찾을 수 없습니다.")

Combined 데이터셋 경로: C:\Users\daboi\Desktop\ai05-level1-project\CombinedDataset3
data.yaml 경로: C:\Users\daboi\Desktop\ai05-level1-project\CombinedDataset3\data_70_15_15_split.yaml

[YOLOv8n] '1단계: Baseline 모델' 학습 시작
목표: Valid mAP75 기준 점수 확보
Ultralytics 8.3.222  Python-3.11.14 torch-2.10.0.dev20251029+cu130 CUDA:0 (NVIDIA GeForce RTX 5070, 12227MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\daboi\Desktop\ai05-level1-project\CombinedDataset3\data_70_15_15_split.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, k